In [31]:
from assignee import *
import pandas as pd
import numpy as np
engine = establish_connection()

## 01 - Sample

In [ ]:
sample_mentions(750)

## 02 - Populate Sample

In [ ]:
with engine.connect() as connection:
    populate_sample(connection=connection, sample_path="data/01 - sample.txt", output_path="data/02 - sample_with_data.csv")

## 03 - Segement Sample
Take the populated sample from Step 02 and split into `n` different files where `n` is the number of hand labelers. Let's assume `n=3` for now.

In [ ]:
segment_sample()